<a href="https://colab.research.google.com/github/Edward-The-Great/DATAset-for-my-finals/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math

# 1) first of.. LOAD AND SPLIT DATA
def load_data_from_url(csv_url):

    df = pd.read_csv(csv_url)
    return df
    #1.2 now i have got the data frame loaded.
    #1.3  actual url provided in section #10

#2) now split to training and testing sets
def split_dataset(df, train_ratio = 0.7):

     n_total = len(df)
     split_index = int(train_ratio * n_total)
     df_train = df.iloc [:split_index].copy()
     df_test = df.iloc [:split_index:].copy()
     return df_train, df_test
     #2.1 so now we got splitted data frame: and parameters!


#3) start KNN classifier: use the parameter above
def KNN(df_train):

     #3.1 creating dictionary out of data
      model_KNN = {
     #3.2 i need a copy not to mess it up
                     'train_data': df_train.copy()
      }
      return model_KNN



#4 okalidi distance between two features
def okalidi_merhak(a_feature, b_feature):
    #4.1 initialize a digit variable
    sq_sum = 0.0

    #4.2 iterate over the features
    for i in range(len(a_feature)):

        #4.3 compute differance
        diff = a_feature[i] - b_feature[i]

        #4.4 accumulate the Squares
        sq_sum += diff * diff

        #4.5 return square root
    return math.sqrt(sq_sum)

#5 KNN prediction function!!
def predict_clas_KNN(new_line, model_KNN, k=3):

  #5.1 access 'train_data' from model_KNN. This is the entire training set
  train_df = model_KNN['train_data']
  dist_labels = []

  #5.2 iterate over rows
  for _, row in train_df.iterrows():
    train_feats = row [:-1].values
    train_label = row['Outcome']

    #5.3 collect (distance, label) pairs
    dist = okalidi_merhak (new_line, train_feats)
    dist_labels.append((dist, train_label))

  #5.4 sort by distance ascending
  dist_labels.sort(key=lambda x: x[0])

  #5.5 get the top k labels
  top_k = dist_labels[:k]

  #5.5 majoroty vote
  count_ones = sum(1 for (d, lab) in top_k if lab == 1)
  count_zeros = k - count_ones
  prediction = 1 if count_ones > count_zeros else 0

  #5.6 returns prediction = 0 or 1
  p1 = count_ones/ float(k)
  p0 = 1- p1
  return prediction, p1, p0






# 6 NAIVE BAYES classifier




#6.1 split df into classes
def separate_by_class(df):
    df_class0 = df[df['Outcome'] == 0].copy()
    df_class1 = df[df['Outcome'] == 1].copy()
    return df_class0, df_class1
    # we got them classes

#6.2 compute mean & variance (ddof=1) for 8 features
def compute_stats_for_class(df_class):
    features = df_class.iloc[:, :-1]
    means = features.mean(axis=0).values
    variances = features.var(axis=0, ddof=1).values
    return means, variances



# 7 Probability density function of a Normal distribution




#7.1 (mean,var) for a single future value x
def gaausian_pdf(x, mean, var):

    #7.2 to avoid zero variance
    eps = 1e-4
    if var < eps:
       var = eps

    # 7.3 standard deviation
    std = math.sqrt(var)
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * var)))
    return (1.0 / (math.sqrt(2 * math.pi) * std)) * exponent

# 8 compute the unnormalized posterior for one class
def posterior_score(row_features, means, variances, prior):

   #8.1  the prior probability for that class
   score = prior
   for i in range(len(row_features)):
      x_i = row_features[i]
      mu_i = means[i]
      var_i = variances[i]

      #8.2 multiply by the product of gusian PDF for each feature
      score *= gaausian_pdf(x_i, mu_i, var_i)
   return score

# 9 training function
def train_naive_bayes(df_train):

    #9.1 splitting rows into df0 (Outcome=0) and df1 (Outcome=1)
    df0, df1 = separate_by_class(df_train)
    n_total = len(df_train)
    n_c0 = len(df0)
    n_c1 = len(df1)

    #9.2 computing the prior probabilities
    prior_c0 = n_c0 / float(n_total)
    prior_c1 = n_c1 / float(n_total)

    #9.3 calculating means/variances for each class
    means0, vars0 = compute_stats_for_class(df0)
    means1, vars1 = compute_stats_for_class(df1)

    #9.4 now i need a dictionary storing all these values
    model_naive_bayes = {
        'means0': means0, 'vars0': vars0, 'prior0': prior_c0,
        'means1': means1, 'vars1': vars1, 'prior1': prior_c1
    }
    #9.5 return the dic
    return model_naive_bayes

#10 predict class for a new data row
def predict_clas_naive_bayes(new_line, model_naive_bayes):

    #10.1  compute the Lo normali posteriors for class 0
    score0 = posterior_score(new_line,
                             model_naive_bayes['means0'],
                             model_naive_bayes['vars0'],
                             model_naive_bayes['prior0'])

    #10.2 Compute the Lo normali posteriors for class 1
    score1 = posterior_score(new_line,
                             model_naive_bayes['means1'],
                             model_naive_bayes['vars1'],
                             model_naive_bayes['prior1'])
    #10.3 ssssssssssssssssssss
    total = score0 + score1
    if total < 1e-15:
      p0 = p1 = 0.5
    else:
      p0 = score0 / total
      p1 = score1 / total

    # 10.4 ssssssssssssssssss
    prediction = 0 if score0 > score1 else 1
    return prediction, p0, p1




# 11 decision tree CLASSifier




#11.1 compute shannon entropy, for a binary labels (0 or 1)
def entropy(samples):
  n_total = len(samples)
  if n_total == 0:
    return 0
  count_ones = sum(samples)
  p1 = count_ones / float(n_total)
  if p1 == 0 or p1==1:
    return 0
  p0 = 1 - p1
  return - (p0 * math.log2(p0) + p1 * math.log2(p1))

#11.2 find the best feature (and threshold = mean)
#  to split the data in a simple ID3-like approach
def best_split(df):
    base_entropy = entropy(df['Outcome'].values)
    n_features = df.shape[1] - 1
    best_gain = 0.0
    best_feature = None
    best_threshold = None

    # 11.3 xxxxxxxxxxx
    for feat_index in range(n_features):
        #11.4 pick treshold = mean of that column
        column_values = df.iloc[:, feat_index].values
        threshold = np.mean(column_values)

        #11.5 split df into left and right by that treshold
        left_df = df[df.iloc[:, feat_index] <= threshold]
        right_df = df[df.iloc[:, feat_index] > threshold]

        # 11.6
        left_entropy = entropy(left_df['Outcome'].values)
        right_entropy = entropy(right_df['Outcome'].values)

        # 11.7
        w_left = len(left_df) / float(len(df))
        w_right = 1 - w_left

        # 11.8
        new_entopy = w_left * left_entropy + w_right * right_entropy
        info_gain = base_entropy - new_entopy

        # 11.9
        if info_gain > best_gain:
            best_gain = info_gain
            best_feature = feat_index
            best_threshold = threshold
    #  11.10
    if best_feature is None or best_gain < 1e-9:
         return None
    return (best_feature, best_threshold)

#12 Recursively build a decision tree
def build_tree(df, depth=0, max_depth=5):

    # 12.1
    outcomes = df['Outcome'].values
    n_ones = sum(outcomes)
    n_zeros = len(df) - n_ones

    #12.2 If pure or empty or max_depth reached => leaf
    if len(df) == 0 or n_ones == 0 or n_zeros == 0 or depth >= max_depth:

        #12.3 majority class
        maj = 1 if n_ones >= n_zeros else 0
        return {'leaf': True, 'outcome': maj}
    # 12.4
    split = best_split(df)
    if split is None:

        #12.5 no good split = leaf
        maj = 1 if n_ones >= n_zeros else 0
        return {'leaf': True, 'outcome': maj}
    feat, thr = split

    #12.6 build subtrees
    left_df = df[df.iloc[:, feat] <= thr]
    right_df = df[df.iloc[:, feat] > thr]

    # 12.7
    left_subtree = build_tree(left_df, depth+1, max_depth)
    right_subtree = build_tree(right_df, depth+1, max_depth)

    # 12.8
    return {
        'leaf': False,
        'feature': feat,
        'threshold': thr,
        'left': left_subtree,
        'right': right_subtree
    }

#13 build a tree from training df.
def train_decision_tree(df_train, max_depth=5):

    #13.1 Returns 'model_dt' which is the root node of the tree.
    root = build_tree(df_train, depth=0, max_depth=max_depth)
    model_dt = {
        'root': root
    }
    return model_dt

#14  Predict a single row's outcome by traversing the tree node.
def predict_one_dt(row_features, node):
    # 14.1
    if node['leaf']:
        return node['outcome']  # 0 or 1
    # 14.2
    feat = node['feature']
    thr = node['threshold']
    val = row_features[feat]

    # 14.3
    if val <= thr:
        return predict_one_dt(row_features, node['left'])
    else:
        return predict_one_dt(row_features, node['right'])

#15 Predict for a single row. We'll only return label, no probability here.
def predict_class_dt(new_row_features, model_dt):

    # 15.1
    root = model_dt['root']
    pred = predict_one_dt(new_row_features, root)
    return pred


#16  EVALUATION (ACCURACY, PRECISION, RECALL, F1)
def evaluate_knn(df, model_knn, k=3):

    # 16.1
    TP = FP = TN = FN = 0
    for _, row in df.iterrows():

        # 16.2
        feats = row[:-1].values
        true_label = row['Outcome']

        # 16.3
        pred_label, _, _ = predict_clas_KNN(feats, model_knn, k)


        if pred_label == 1 and true_label == 1:
            TP += 1
        elif pred_label == 1 and true_label == 0:
            FP += 1
        elif pred_label == 0 and true_label == 0:
            TN += 1
        elif pred_label == 0 and true_label == 1:

            # 16.4
            FN += 1

    # 16.5
    total = TP+FP+TN+FN
    accuracy = (TP+TN)/total if total>0 else 0
    precision = TP/(TP+FP) if (TP+FP)>0 else 0
    recall = TP/(TP+FN) if (TP+FN)>0 else 0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0
    return accuracy, precision, recall, f1

#17 Evaluate Naive Bayes = returns (accuracy, precision, recall, f1)
def evaluate_nb(df, model_nb):

    # 17.1
    TP = FP = TN = FN = 0

    # 17.2
    for _, row in df.iterrows():

        # 17.3
        feats = row[:-1].values
        true_label = row['Outcome']

        # 17.4
        pred_label, _, _ = predict_clas_naive_bayes(feats, model_nb)

        # 17.5
        if pred_label == 1 and true_label == 1:
            TP += 1
        elif pred_label == 1 and true_label == 0:
            FP += 1
        elif pred_label == 0 and true_label == 0:
            TN += 1
        elif pred_label == 0 and true_label == 1:

            # 17.6
            FN += 1

    # 17.7
    total = TP+FP+TN+FN

    accuracy = (TP+TN)/total if total>0 else 0
    precision = TP/(TP+FP) if (TP+FP)>0 else 0
    recall = TP/(TP+FN) if (TP+FN)>0 else 0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0
    return accuracy, precision, recall, f1

#18 Evaluate Decision Tree = returns (accuracy, precision, recall, f1)
def evaluate_dt(df, model_dt):

    # 18.1
    TP = FP = TN = FN = 0

    # 18.2
    for _, row in df.iterrows():

        # 18.3
        feats = row[:-1].values
        true_label = row['Outcome']

        # 18.4
        pred_label = predict_class_dt(feats, model_dt)

        # 18.5
        if pred_label == 1 and true_label == 1:
            TP += 1
        elif pred_label == 1 and true_label == 0:
            FP += 1
        elif pred_label == 0 and true_label == 0:
            TN += 1
        elif pred_label == 0 and true_label == 1:

            # 18.6
            FN += 1

    # 18.7
    total = TP+FP+TN+FN

    # 18.8
    accuracy = (TP+TN)/total if total>0 else 0
    precision = TP/(TP+FP) if (TP+FP)>0 else 0
    recall = TP/(TP+FN) if (TP+FN)>0 else 0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0
    return accuracy, precision, recall, f1


# 19 MAIN

#19.1 suplying the url
csv_url = "https://raw.githubusercontent.com/Edward-The-Great/DATAset-for-my-finals/refs/heads/main/diabetes.csv"

#19.2 Load full dataset
df_full = load_data_from_url(csv_url)

#19.3 Split train/test
df_train, df_test = split_dataset(df_full, train_ratio=0.7)

#19.4 KNN
model_knn = KNN(df_train)

# 19.5
acc_knn_tr, prec_knn_tr, rec_knn_tr, f1_knn_tr = evaluate_knn(df_train, model_knn, k=3)
acc_knn_te, prec_knn_te, rec_knn_te, f1_knn_te = evaluate_knn(df_test, model_knn, k=3)

# 20 results!!!!!

# 20.1 KNN
print("                results for KNN  ")
print(f"Train = Accuracy={acc_knn_tr:.3f}, Precision={prec_knn_tr:.3f}, Recall={rec_knn_tr:.3f}, F1={f1_knn_tr:.3f}")
print(f"Test  = Accuracy={acc_knn_te:.3f}, Precision={prec_knn_te:.3f}, Recall={rec_knn_te:.3f}, F1={f1_knn_te:.3f}")

# 20.2 Naive Bayes
model_nb = train_naive_bayes(df_train)
acc_nb_tr, prec_nb_tr, rec_nb_tr, f1_nb_tr = evaluate_nb(df_train, model_nb)
acc_nb_te, prec_nb_te, rec_nb_te, f1_nb_te = evaluate_nb(df_test, model_nb)

print("\n            results for Naive Bayes ")
print(f"Train = Accuracy={acc_nb_tr:.3f}, Precision={prec_nb_tr:.3f}, Recall={rec_nb_tr:.3f}, F1={f1_nb_tr:.3f}")
print(f"Test  = Accuracy={acc_nb_te:.3f}, Precision={prec_nb_te:.3f}, Recall={rec_nb_te:.3f}, F1={f1_nb_te:.3f}")

# 20.3 Decision Tree
model_dt = train_decision_tree(df_train, max_depth=5)
acc_dt_tr, prec_dt_tr, rec_dt_tr, f1_dt_tr = evaluate_dt(df_train, model_dt)
acc_dt_te, prec_dt_te, rec_dt_te, f1_dt_te = evaluate_dt(df_test, model_dt)

print("\n           results for Decision Tree ")
print(f"Train = Accuracy ={acc_dt_tr:.3f}, Precision={prec_dt_tr:.3f}, Recall={rec_dt_tr:.3f}, F1={f1_dt_tr:.3f}")
print(f"Test  = Accuracy ={acc_dt_te:.3f}, Precision={prec_dt_te:.3f}, Recall={rec_dt_te:.3f}, F1={f1_dt_te:.3f}")

# end of code


                results for KNN  
Train = Accuracy=0.844, Precision=0.814, Recall=0.720, F1=0.764
Test  = Accuracy=0.844, Precision=0.814, Recall=0.720, F1=0.764

            results for Naive Bayes 
Train = Accuracy=0.767, Precision=0.693, Recall=0.608, F1=0.648
Test  = Accuracy=0.767, Precision=0.693, Recall=0.608, F1=0.648

           results for Decision Tree 
Train = Accuracy =0.790, Precision=0.711, Recall=0.677, F1=0.694
Test  = Accuracy =0.790, Precision=0.711, Recall=0.677, F1=0.694
